# L5、L6-Pytorch数据加载

### 1. Dataset,Dataloader,Data

+ `data`: 垃圾，需要挑出有用的数据
+ `dataset`: 提供一种方式去获取数据及其label, 会告诉我们如何去获取每一个数据及其label，会告诉我们总共有多少数据
+ `dataloader`: 提供一种方法能让我们对导入的数据进行处理

In [2]:
from torch.utils.data import Dataset

In [3]:
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(builtins.object)
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |  
 |  __getitem__(self, index)
 |  
 |  --------------------------------------------------

**__xxx__，这种变量定义的格式一般都是说明该变量是标准，不能被修改。**

### 2.自定义类读取数据集：

**torch.utils.data.Dataset**
**要自定义自己的 Dataset 类，需要重载两个方式，【__len__】、【__getitem__】**
+ 【__len__】返回数据集的大小
+ 【__getitem__】实现索引数据集中的某一个元素

In [5]:
from PIL import Image

In [10]:
# 打开图片
image_path = "./dataset/train/ants/0013035.jpg"
img = Image.open(image_path)

In [12]:
# 输出大小以及打开图片
img.size
img.show()

In [19]:
# os库
import os
root_dir = "./dataset/train"
label_dir = "ants"
path = os.path.join(root_dir, label_dir)                   # 将前后两个目录用'/'分隔开来
print(path )
print("----------------------------------------")
image_path = os.listdir(path)                                      # 将这个路径下的文件名做成一个列表
print(image_path )
print("----------------------------------------")
print(image_path[0])

./dataset/train/ants
----------------------------------------
['2019439677_2db655d361.jpg', 'termite-vs-ant.jpg', '5650366_e22b7e1065.jpg', '6240338_93729615ec.jpg', '150801171_cd86f17ed8.jpg', '474806473_ca6caab245.jpg', '20935278_9190345f6b.jpg', '1808777855_2a895621d7.jpg', '424873399_47658a91fb.jpg', '522163566_fec115ca66.jpg', 'kurokusa.jpg', '2288450226_a6e96e8fdf.jpg', '382971067_0bfd33afe0.jpg', '162603798_40b51f1654.jpg', '998118368_6ac1d91f81.jpg', '460874319_0a45ab4d05.jpg', '1924473702_daa9aacdbe.jpg', '334167043_cbd1adaeb9.jpg', 'formica.jpeg', '512863248_43c8ce579b.jpg', '484293231_e53cfc0c89.jpg', '531979952_bde12b3bc0.jpg', '533848102_70a85ad6dd.jpg', '226951206_d6bf946504.jpg', '938946700_ca1c669085.jpg', '2265825502_fff99cfd2d.jpg', '684133190_35b62c0c1d.jpg', 'hormiga_co_por.jpg', '384191229_5779cf591b.jpg', '1286984635_5119e80de1.jpg', '45472593_bfd624f8dc.jpg', '1917341202_d00a7f9af5.jpg', '470127037_513711fd21.jpg', '892108839_f1aad4ca46.jpg', '339670531_94b75ae47

In [27]:
from torch.utils.data import Dataset
from PIL import Image
import os

class MyData(Dataset):                                                                # 继承自Dataset这个类
    
    def __init__(self, root_dir, label_dir):                               # 一般都是定义一些全局变量
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.image_path = os.listdir(self.path)
        
    def __getitem__(self, idx):
        img_name = self.image_path[idx]
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label
        
    def __len__(self):                                                                     # 返回数据集的长度
        return len(self.image_path)

In [25]:
root_dir = "./dataset/train"
ants_label_dir = "ants"
ants_dataset = MyData(root_dir, ants_label_dir)

#### 2.1查看蚂蚁数据集：

In [37]:
img, label = ants_dataset[0]
img.show()

❓: **为什么ants_dataset[0]是直接调用__getitem__方法?**    
   
GPT：   
在Python中，当你使用方括号 [] 语法访问一个对象时，Python 会自动调用该对象的特殊方法 __getitem__。
这是Python数据模型的一部分，它允许对象实现容器类型的行为，如列表或字典。 MyData 类继承自 torch.utils.data.Dataset，后者定义了 **getitem** 和 **len** 这样的特殊方法，使其子类能够表现得像一个序列（如列表）。这样，当你对这个数据集对象进行索引访问，如 ants_dataset[0]，Python解释器就会自动调用 MyData 类的 **getitem** 方法，并传递索引 0 作为参数。 **getitem** 方法通常用于检索序列中的元素。在PyTorch的 Dataset 类中，这个方法被用来加载和返回数据集中特定索引的数据样本。这就是为什么调用 ants_dataset[0] 返回数据集中第一个样本，即索引为0的样本。这也是您能够通过索引轻松访问数据集中任意样本的原因。 具体到 MyData 类，当 ants_dataset[0] 被调用时，以下这段代码将执行：
```python
def __getitem__(self, idx):
    img_name = self.image_path[idx]  # 获取图像名称
    img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)  # 创建完整的图像路径
    img = Image.open(img_item_path)  # 加载图像
    label = self.label_dir  # 这里应该是提取特定图像的标签，但在提供的代码中只是简单地返回了标签目录的名称
    return img, label
```
因此，它返回了第一个图像和一个标签（在你的代码中标签是目录名 "ants"，但通常应该是图像的实际标签）。

#### 2.2 查看蜜蜂数据集：

In [38]:
bees_label_dir = "bees"
bees_dataset = MyData(root_dir, bees_label_dir)
bees_dataset[2]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x7EFBF82AD908>,
 'bees')

In [39]:
img, label = bees_dataset[2]
img.show()

#### 2.3 数据集可以相加:

现实中训练模型常常遇到数据集不足情况，有时候需要仿制数据集，所以需要数据集合的拼接操作。


In [40]:
train_dataset = ants_dataset + bees_dataset

In [42]:
len(bees_dataset)

121

In [43]:
len(ants_dataset)

124

In [44]:
len(train_dataset)

245